# Agenda: Grouping and pivot tables

1. What is grouping?
2. Simple grouping
3. Choosing aggregate functions
4. Group on more than one column
5. Retrieving more than one column
6. Multiple aggregations
7. Grouping on dates and times
8. Grouper
9. Pivot tables

# What is grouping?

If we have a numeric column, we can run an aggregation (e.g., `mean` or `sum` or `count`) on it.  

If we have two columns, one numeric and one categorical, then we can ask to run an aggregation function on all of the columns for a particular value of the categorical column. 

Pandas provides us with an easy to to perform the aggregation function for each subset of the table, once for each value of the categorical.